<a href="https://colab.research.google.com/github/alicelindel3/ibm5100/blob/main/1129/week7_home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework7
時系列データを含むデータベースファイル (data.sqlite3) に含まれるデータについて、goods_genre_id ごとに price の平均値を求めてください。時系列データには goods_genre_id の情報は含まれていないので、複数のテーブルを組み合わせて処理する必要があります。  
データの詳細は以下を参照してください。

### データ一覧
今回与えられるデータは data.sqlite3 という sqlite3 形式のデータです。
以下の 2 つのテーブルが含まれています。
* data ... 時系列データ
* goods ... 商品に関する補足情報

### カラム情報一覧
データに含まれるカラムの情報は以下です。
* index ... (店舗, 商品) の各組み合わせに対して割り振られるID
* store_id ... 店舗ID
* goods_id ... 商品ID
* goods_genre_id ... 商品カテゴリID
* units_sold_day ... 商品の一日の売り上げ個数
* units_sold_month ... 商品のひと月の売り上げ個数
* price ... 商品の現在の値段
* yy-mm-dd ... 日付 (yy/mm/dd形式)
* num_month ... 便宜上各月にわりふられた値 (平成25年1月=1, 平成25年2月=2,...,平成27年9月=33)
* goods_name ... 商品名
* store_name ... 店舗名
* goods_genre_name ... 商品カテゴリ名

### 注意事項
- データ処理はSQLiteでの操作を推奨します。補足資料「SQLite_manual.pdf」「SQLite_manual.ipynb」を参照ください。
- SQLiteのインストール/ダウンロードが上手くいかない場合、python上での操作も可能です。補足資料「csv_export_python.pdf」「csv_export_python.ipynb」を参照ください。
- 一部の処理に時間がかかる場合があります。

### 提出/採点
回答は**Omnicampus**上で提出してください。提出するデータの形式は以下のような**csvファイル**となります。
 
goods_genre_id|avg_price
---|---
1|25000
3|26500
4|25000
…|…
196|20000
200|27000


提出ファイルがこの形式に準じていなかった場合にはスコアが1000となるので、その際はファイル形式を確認しましょう。  

In [1]:
import sqlite3
import csv

In [2]:
# Google Colaboratoryで作業する場合に実行してください．  
# データの読み込み方が不明な場合は，別途配布している「GoogleColabにおけるドライブのマウントの仕方」と「Google Drive上でのフォルダ構造の説明」のpdfファイルをご覧ください．
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# データベースファイルのパスを渡してください
dbpath = '/content/drive/My Drive/Colab Notebooks/data.sqlite3'

In [4]:
conn = sqlite3.connect(dbpath)

In [5]:
cur = conn.cursor()

In [6]:
# テーブル名の確認
cur.execute('select name from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('data',)
('goods',)


In [7]:
# カラム名(CREATE TABLE文)の確認
cur.execute('select sql from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)',)
('CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)',)


In [8]:
# 全情報の確認 (type, name, tbl_name, rootpage, sql)
cur.execute('select * from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('table', 'data', 'data', 2, 'CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)')
('table', 'goods', 'goods', 12, 'CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)')


In [ ]:
cur.execute('select * from data')
print(cur.fetchone()) #1レコード目の取得
print(cur.fetchone()) #2レコード目の取得

('25-01-29', '-33', '43', '17718', '44943', '2')
('25-01-08', '-33', '32', '8648', '14949', '1')


In [12]:
cur.execute('select * from data limit 10')
for row in cur.fetchall():
  print(row)

('25-01-29', '-33', '43', '17718', '44943', '2')
('25-01-08', '-33', '32', '8648', '14949', '1')
('25-01-16', '-33', '53', '10168', '30158', '1')
('25-01-23', '-33', '31', '1005', '9922', '1')
('25-01-12', '-33', '27', '19861', '14988', '0')
('25-01-11', '-33', '26', '18473', '19768', '1')
('25-01-09', '-33', '51', '6096', '170492', '1')
('25-01-04', '-33', '7', '13902', '29793', '1')
('25-01-10', '-33', '23', '17213', '15099', '1')
('25-01-20', '-33', '47', '15105', '129142', '1')


In [ ]:
cur.execute('select * from data a join goods b on a.goods_id=b.goods_id')
for row in cur.fetchall():
  print(row)

('25-01-29', '-33', '43', '17718', '44943', '2', 'Receiving cash for 1C-line', '17718', '80')
('25-01-08', '-33', '32', '8648', '14949', '1', 'Barbie as the Princess and pop stars (region)', '8648', '41')
('25-01-16', '-33', '53', '10168', '30158', '1', 'Vysotsky Vladimir theatrical sketch on Tagansky 3DVD topics', '10168', '60')
('25-01-23', '-33', '31', '1005', '9922', '1', '3D Action Puzzle "Technique" racing machine', '1005', '68')
('25-01-12', '-33', '27', '19861', '14988', '0', 'Three from Buttermilk (Region)', '19861', '41')
('25-01-11', '-33', '26', '18473', '19768', '1', 'Sat. Dance Paradise 29', '18473', '56')
('25-01-09', '-33', '51', '6096', '170492', '1', 'Rayman Origins [PS Vita, Russian documentation]', '6096', '23')
('25-01-04', '-33', '7', '13902', '29793', '1', 'Leps Grigory Full Throttle (firms).', '13902', '56')
('25-01-10', '-33', '23', '17213', '15099', '1', 'MIDNIGHT IN PARIS', '17213', '41')
('25-01-20', '-33', '47', '15105', '129142', '1', 'microrobot Spider', 

In [14]:
cur.execute('select goods_genre_id, avg(price) as avg_price from data a join goods b on a.goods_id=b.goods_id group by goods_genre_id order by cast (goods_genre_id as integer)')
for row in cur.fetchall():
  print(row)

('3', 168482.4)
('4', 47476.5)
('5', 129025.0)
('6', 39211.333333333336)
('7', 158876.2)
('12', 1381663.0)
('15', 1197012.5)
('16', 1315036.0)
('19', 29577.0)
('20', 154165.75)
('21', 248176.0)
('22', 52100.066666666666)
('23', 167440.53846153847)
('24', 164675.7037037037)
('26', 71894.57142857143)
('29', 63498.39473684211)
('30', 129826.33333333333)
('31', 42573.60625)
('32', 41988.6)
('34', 120571.0)
('36', 222425.0)
('38', 42881.25)
('39', 108544.72222222222)
('41', 27312.9203187251)
('42', 96667.75)
('44', 28968.470588235294)
('46', 20584.5)
('50', 34330.857142857145)
('56', 26939.964285714286)
('58', 26590.416666666668)
('59', 92398.5)
('60', 45314.42857142857)
('61', 171138.33333333334)
('62', 79763.0)
('63', 67526.75)
('64', 44575.181818181816)
('65', 99287.0625)
('66', 30284.083333333332)
('67', 13455.6)
('68', 47454.0)
('70', 81622.5)
('71', 29752.75)
('73', 142723.16666666666)
('74', 95758.0)
('76', 162571.72222222222)
('77', 149070.0)
('78', 24784.666666666668)
('80', 44943.

In [15]:
#テーブルから特定の列を取得
cur.execute('select goods_genre_id, avg(price) as avg_price from data a join goods b on a.goods_id=b.goods_id group by goods_genre_id order by cast (goods_genre_id as integer)')
with open('out.csv', 'w', newline='') as csvfile:
  csv_writer = csv.writer(csvfile)
  csv_writer.writerow([i[0] for i in cur.description])
  csv_writer.writerows(cur)

In [16]:
# 処理を確定
conn.commit()

In [17]:
cur.close()

In [18]:
conn.close()

In [19]:
import pandas as pd

In [20]:
pd.read_csv('out.csv', sep=',')

,goods_genre_id,avg_price
0,3,1.684824e+05
1,4,4.747650e+04
2,5,1.290250e+05
3,6,3.921133e+04
4,7,1.588762e+05
5,12,1.381663e+06
6,15,1.197012e+06
7,16,1.315036e+06
8,19,2.957700e+04
9,20,1.541658e+05


In [21]:
from google.colab import files
files.download('out.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>